In [ ]:
import os
import random
import shutil

In [ ]:
# Diretórios de origem e destino dos arquivos
objeto_cortante_images_path = r"C:\YOLO\images"
objeto_cortante_labels_path = r"C:\YOLO\labels"
neutro_images_path = r"C:\YOLO\neutro_image"
output_dir = r"C:\YOLO\images_treinamento"

In [ ]:
# Divisão dos dados em treino, validação e teste
train_ratio = 0.85
val_ratio = 0.0
test_ratio = 0.15

In [ ]:
# Função para dividir os arquivos em treino, validação e teste
# images_path: diretório das imagens
# labels_path: diretório dos labels
# output_dir: diretório de saída
# train_ratio: proporção de imagens para treino
# val_ratio: proporção de imagens para validação
# class_name: nome da classe
# max_images: número máximo de imagens para processar
# has_labels: se os arquivos possuem labels
def split_data(images_path, labels_path, output_dir, train_ratio, val_ratio, class_name, max_images=None, has_labels=True): 
    images = sorted(os.listdir(images_path))
    
    if has_labels:
        labels = sorted(os.listdir(labels_path))

        images = [img for img in images if img.replace('.jpg', '.txt') in labels]
    
    if max_images is not None:
        images = images[:max_images]

    random.shuffle(images)

    total_images = len(images)
    train_count = int(total_images * train_ratio)
    val_count = int(total_images * val_ratio)

    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    def move_files(image_list, subset):
        image_subset_dir = os.path.join(output_dir, "images", subset)
        os.makedirs(image_subset_dir, exist_ok=True)

        if has_labels:
            label_subset_dir = os.path.join(output_dir, "labels", subset)
            os.makedirs(label_subset_dir, exist_ok=True)
        
        for img in image_list:
            shutil.copy(os.path.join(images_path, img), os.path.join(image_subset_dir, img))
            
            if has_labels:
                label_file = img.replace('.jpg', '.txt')
                shutil.copy(os.path.join(labels_path, label_file), os.path.join(label_subset_dir, label_file))

    move_files(train_images, "train")
    move_files(val_images, "val")
    move_files(test_images, "test")
    
    print(f"{class_name} -> Train: {len(train_images)}, Val: {len(val_images)}, Test: {len(test_images)}")

In [ ]:
# Função para contar a quantidade de arquivos em um diretório
# folder_path: diretório
# file_extensions: extensões dos arquivos
def count_files_in_folder(folder_path, file_extensions=None):
    if not os.path.exists(folder_path):
        print(f"Erro: O diretório '{folder_path}' não existe.")
        
        return 0
    
    all_files = os.listdir(folder_path)
    
    if file_extensions:
        all_files = [f for f in all_files if f.lower().endswith(file_extensions)]
    
    return len(all_files)

In [ ]:
num_images = count_files_in_folder(objeto_cortante_images_path)
print(f"Total de imagens na pasta: {num_images}")

In [ ]:
split_data(
    images_path=objeto_cortante_images_path,
    labels_path=objeto_cortante_labels_path,
    output_dir=output_dir,
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    class_name="Objeto Cortante",
    max_images=100000,
    has_labels=True
)

In [ ]:
num_images = count_files_in_folder(neutro_images_path)
print(f"Total de imagens na pasta: {num_images}")

In [ ]:
split_data(
    images_path=neutro_images_path,
    labels_path=None,
    output_dir=output_dir,
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    class_name="Neutro",
    max_images=4000,
    has_labels=False
)